# `Pre-processing`

## 1 Setup

### 1.1 Libraries

In [45]:
import pandas as pd
import numpy as np

### 1.2 Paths

In [46]:
DEPRESSION_PATH             = '/workspaces/depressao-ibge-pns/data/depression.csv'
DEPRESSION_DICTIONARY_PATH  = '/workspaces/depressao-ibge-pns/data/depression-dictionary.csv'

### 1.3 Dataframes

In [47]:
DEPRESSION            = pd.read_csv(DEPRESSION_PATH, header=[0, 1])
DEPRESSION_DICTIONARY = pd.read_csv(DEPRESSION_DICTIONARY_PATH)

In [48]:
DEPRESSION.head()

,V0001,V0026,V0031,C006,C00703,C008,C009,C01001,C013,C017,...,V033,V034,V03501,V03502,V03503,V036,V037,V038,V039,Y008
,Unidade da Federação,Tipo de situação censitária,Tipo de área,Sexo,Ano de nascimento,Idade do morador na data de referência,Cor ou raça,Cônjuge ou companheiro(a) mora em nesse domicílio.,Cônjuge ou companheiro(a) mora em outro domicílio.,___já viveu com cônjuge ou companheiro (a) antes?,...,Onde isso ocorreu?,"Nos últimos doze meses, você deixou de realizar quaisquer de suas atividades habituais (trabalhar, realizar afazeres domésticos, ir à escola etc.) por causa desse ato","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Hematomas, cortes, fraturas, queimaduras ou outras lesões físicas ou ferimentos?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Medo, tristeza, desânimo, dificuldade para dormir, ansiedade, depressão ou outras consequências psicológicas?","Esse(s) ato(s) sexual(is) forçado(s) gerou(aram) alguma consequência para sua saúde, tais como: Doença sexualmente transmissível ou gravidez indesejada?","Por causa desta (s) consequência (s), você procurou algum atendimento de saúde","Por causa desta (s) consequência (s), você recebeu algum atendimento de saúde",Onde foi realizado este atendimento de saúde,"Por causa desta (s) consequência (s), você precisou ser internado por 24 horas ou mais",Qual é a sua orientação sexual?
0,11,1,1,2.0,1963.0,55.0,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,11,1,1,1.0,1950.0,69.0,4.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11,1,1,1.0,1987.0,31.0,2.0,2.0,2.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,1,1,1.0,9999.0,9.0,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1,1,2.0,9999.0,6.0,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
DEPRESSION_DICTIONARY.head()

,ModuloParteAnotacao,Posicao,Tamanho,Codigo,Numero,Descricao,Dtype,Tipo,Classe,Include,Felipe,Diego
0,Parte 1 - Identificação e Controle,1.0,2.0,V0001,NaN,Unidade da Federação,category,11,Rondônia,True,True,True
1,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,12,Acre,True,True,True
2,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,13,Amazonas,True,True,True
3,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,14,Roraima,True,True,True
4,Parte 1 - Identificação e Controle,NaN,NaN,V0001,NaN,Unidade da Federação,NaN,15,Pará,True,True,True


## 2 Implicit `NaNs` `TODO`: [Issue - Remove useless classes from depression.csv #4](https://github.com/feed0/depressao-ibge-pns/issues/4)
> Useless classes such as '09 - Ignored' become NaNs

>Here we define the `depression_explicit`: pd.DataFrame(),<br>
>which will drop useless classes such as **'09 - I don't know'**

In [ ]:
# Here, `9999.0` means "Not answered"
DEPRESSION.C00703.value_counts(ascending=False).head(3)

Ano de nascimento
9999.0               16342
2000.0                4434
2004.0                4225
Name: count, dtype: int64

In [93]:
# These are the Tipos we need to annull

tipos_to_remove = DEPRESSION_DICTIONARY[
    (DEPRESSION_DICTIONARY.Tipo.notna())
    & (DEPRESSION_DICTIONARY.Include == False)
]

display(
    tipos_to_remove.shape,
    tipos_to_remove.head(3)
)

(36, 12)

,ModuloParteAnotacao,Posicao,Tamanho,Codigo,Numero,Descricao,Dtype,Tipo,Classe,Include,Felipe,Diego
37,Módulo C - Características gerais dos moradores,NaN,NaN,C00703,C7,Ano de nascimento,NaN,9999,Não informado,False,False,False
46,Módulo C - Características gerais dos moradores,NaN,NaN,C009,C9,Cor ou raça,NaN,9,Ignorado,False,False,False
50,Módulo C - Características gerais dos moradores,NaN,NaN,C01001,C10a,Cônjuge ou companheiro(a) mora em nesse domicí...,NaN,9,Ignorado,False,False,False


In [95]:
# In depression['C00703']   replace 9999    with np.nan
# In depression['C009']     replace 9       with np.nan
# ...
# In depression['Q109']  replace 9       with np.nan

depression_explicit = DEPRESSION.copy()

for codigo in tipos_to_remove.Codigo:
    
    print(f'    [CODIGO]: {codigo} ==================================================')
    print(f'    [BEFORE]:\n{depression_explicit[codigo].value_counts(ascending=False).head(3)}\n')

    code_tipos = tipos_to_remove.Tipo[tipos_to_remove.Codigo == codigo].values.astype(int)
    print(f'    [CODE TIPOS]: === {code_tipos} ===')

    # Replace values in the specific column (convert array to list)
    depression_explicit[codigo] = depression_explicit[codigo].replace(code_tipos.tolist(), np.nan)

    print(f'    [AFTER]:\n{depression_explicit[codigo].value_counts(ascending=False).head(2)}\n')

    [CODIGO]: C00703 ==================================================
    [BEFORE]:
Ano de nascimento
9999.0               16342
2000.0                4434
2004.0                4225
Name: count, dtype: int64

    [CODE TIPOS]: === [9999] ===
    [AFTER]:
Ano de nascimento
2000.0               4434
2004.0               4225
Name: count, dtype: int64

    [CODIGO]: C009 ==================================================
    [BEFORE]:
Cor ou raça
4.0            148273
1.0             99019
2.0             28304
Name: count, dtype: int64

    [CODE TIPOS]: === [9] ===
    [AFTER]:
Cor ou raça
4.0            148273
1.0             99019
Name: count, dtype: int64

    [CODIGO]: C01001 ==================================================
    [BEFORE]:
Cônjuge ou companheiro(a) mora em nesse domicílio.
1.0                                                   122912
2.0                                                   119532
Name: count, dtype: int64

    [CODE TIPOS]: === [9] ===
    [AFTER]:
C

## `TODO`: Class `mapping`
depression_without_implicit C00703

## `TODO`: D`types`
Dtypes are already included at DEPRESSION_DICTIONARY['Dtype'] for each Codigo attribute

In [102]:
# SELECT * FROM DEPRESSION_DICTIONARY
# WHERE Codigo == 'V03501'
DEPRESSION_DICTIONARY[DEPRESSION_DICTIONARY.Codigo == 'V03501']

,ModuloParteAnotacao,Posicao,Tamanho,Codigo,Numero,Descricao,Dtype,Tipo,Classe,Include,Felipe,Diego
391,Módulo V - Violência (Para pessoas de 18 ano...,1275.0,1.0,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,bool,1,NaN,True,True,NaN
392,Módulo V - Violência (Para pessoas de 18 ano...,NaN,NaN,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,NaN,2,NaN,True,True,NaN
393,Módulo V - Violência (Para pessoas de 18 ano...,NaN,NaN,V03501,V35,Esse(s) ato(s) sexual(is) forçado(s) gerou(ara...,NaN,NaN,Não aplicável,False,False,False


## `TODO` Drop columns without class descriptions
Codigo `V03501` is not correctly described in the dictionary-pns-2019-cleaned.csv file.

In [ ]:
# V03501
PNS_DICTIONARY_PATH[PNS_DICTIONARY_PATH.Codigo == 'V03501']

In [ ]:
# Original shape
print(PNS_DICTIONARY_PATH.shape)

# Drop Codigo 'V03501'
PNS_DICTIONARY_PATH = PNS_DICTIONARY_PATH[PNS_DICTIONARY_PATH.Codigo != 'V03501']

# Shape after dropping
print(PNS_DICTIONARY_PATH.shape)